# 🚀 Voicet: AI-Powered Voice Translation & TTS (Standard Edition)

This notebook allows you to run the **Voicet** project on Google Colab using its GPU/CPU resources.

### Steps to Run:
1. **Setup Environment**: Install system dependencies and clone the repository.
2. **Manual Dependency Setup**: Manually clone `VAKYANSH_TTS`.
3. **Install Dependencies**: Install Python packages.
4. **Download Models**: Fetch the high-quality TTS models.
5. **Start Application**: Run the Flask app and access it via a public URL.

## 1. Setup Environment
First, we need to install `ffmpeg` and `sox` for audio processing and clone the repository.

In [ ]:
# Always start from /content to avoid 'getcwd' ghosting errors
%cd /content

# Install system dependencies
!apt-get update && apt-get install -y ffmpeg sox

# Force reset current directory if it was deleted
import os
if os.path.exists('/content/Voicet'):
    !rm -rf /content/Voicet

# Clone the main repository
REPO_URL = "https://github.com/Ujjawalmaurya/Voicet.git" 
!git clone {REPO_URL}

## 2. Manual Dependency Setup
Manually clone `VAKYANSH_TTS` into the correct location to avoid submodule errors.

In [ ]:
%cd /content/Voicet

# Remove empty submodule dir if created
!rm -rf /content/Voicet/VAKYANSH_TTS

# Clone VAKYANSH_TTS manually
!git clone https://github.com/Ujjawalmaurya/vakyansh-tts.git VAKYANSH_TTS

## 3. Install Python Dependencies
Installing all necessary libraries into the current notebook environment.

In [ ]:
%cd /content/Voicet
import sys

# Using --ignore-installed to bypass Colab's distutils package lock (e.g., blinker)
!{sys.executable} -m pip install --ignore-installed -r requirements.txt
!{sys.executable} -m pip install pyngrok yt-dlp
#!{sys.executable} -m pip install localtunnel # Alternative to ngrok

## 4. Download TTS Models
Run the provided script to download all necessary language models.

In [ ]:
%cd /content/Voicet
!chmod +x setup_models.sh
!./setup_models.sh

## 5. Start Voicet Application
We use `ngrok` to create a public tunnel. We've added a cleanup step to ensure old tunnels are closed.

> **Note:** Get your Ngrok Authtoken from [dashboard.ngrok.com](https://dashboard.ngrok.com/get-started/your-authtoken).

In [ ]:
from pyngrok import ngrok
import os
import sys

# --- 1. Cleanup old ngrok sessions ---
print("🧹 Cleaning up existing ngrok tunnels...")
try:
    tunnels = ngrok.get_tunnels()
    for t in tunnels:
        print(f"  Closing {t.public_url}")
        ngrok.disconnect(t.public_url)
    ngrok.kill()
except Exception as e:
    print(f"  No existing tunnels found or error: {e}")

# --- 2. Setup new tunnel ---
# @markdown Enter your Ngrok Authtoken here
NGROK_AUTH_TOKEN = "" # @param {type:"string"}
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("🌐 Starting Flask application...")
try:
    public_url = ngrok.connect(5000)
    print(f"🔗 *Public URL:* {public_url}")
except Exception as e:
    print(f"❌ Ngrok failed: {e}")
    print("💡 TIP: If you hit the tunnel limit again, try: 1. Sign in to ngrok dashboard and delete endpoints. 2. Use Localtunnel instead.")

# Set FLASK_APP environment variable and change dir
os.environ['FLASK_APP'] = 'project'
%cd /content/Voicet/Voicet

# Run Flask
!{sys.executable} -m flask run --host=0.0.0.0 --port=5000